In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Settings
pd.set_option('display.max_columns', None)
sns.set(style="darkgrid")
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

def load_data(file_path):
    """Loads dataset and handles potential errors."""
    try:
        data = pd.read_csv(file_path)
        print(f"Data loaded successfully from {file_path}")
        return data
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
    except pd.errors.EmptyDataError:
        print(f"Error: No data found in the file at {file_path}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    return None

def clean_data(df):
    """Cleans dataset by handling missing values and redundant columns."""
    try:
        # Drop redundant columns
        redundant_cols = ['num_outbound_cmds'] if 'num_outbound_cmds' in df.columns else []
        df.drop(columns=redundant_cols, inplace=True, errors='ignore')

        # Replace infinite values with NaN
        df.replace([np.inf, -np.inf], np.nan, inplace=True)

        # Fill missing values
        for col in df.columns:
            if df[col].dtype == 'object':
                df[col] = df[col].fillna(df[col].mode()[0])  # Fill with mode for categorical columns
            else:
                df[col] = df[col].fillna(df[col].median())  # Fill with median for numerical columns

        print("Data cleaning completed.")
        return df
    except Exception as e:
        print(f"Error during data cleaning: {e}")
    return df

def scale_features(df):
    """Applies feature scaling to numerical columns."""
    numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
    scaler = StandardScaler()
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
    return df

def encode_categorical(df):
    """Encodes categorical variables using Label Encoding."""
    encoder = LabelEncoder()
    categorical_columns = df.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        df[col] = encoder.fit_transform(df[col])
    return df

def feature_selection(df_x, df_y):
    """Performs feature selection using Recursive Feature Elimination (RFE)."""
    rfc = RandomForestClassifier()
    rfe = RFE(rfc, n_features_to_select=15)
    rfe.fit(df_x, df_y)
    selected_features = df_x.columns[rfe.support_]
    return selected_features

def train_classifiers(X_train, Y_train, X_test, Y_test):
    """Trains multiple classifiers and evaluates their performance."""
    models = {
        'KNN': KNeighborsClassifier(n_jobs=-1),
        'Logistic Regression': LogisticRegression(n_jobs=-1, random_state=0),
        'Naive Bayes': BernoulliNB(),
        'Decision Tree': DecisionTreeClassifier(),
        'SVM': SVC(kernel='linear', random_state=0)
    }

    accuracies = {}
    for name, model in models.items():
        model.fit(X_train, Y_train)
        score = model.score(X_test, Y_test)
        accuracies[name] = score
        print(f"{name} Accuracy: {score:.4f}")

    # Plot classifier accuracies
    plt.figure(figsize=(10, 6))
    sns.barplot(x=list(accuracies.keys()), y=list(accuracies.values()), palette="viridis")
    plt.title("Classifier Accuracies")
    plt.xlabel("Classifier")
    plt.ylabel("Accuracy")
    plt.ylim(0, 1)
    plt.show()

def plot_feature_importance(df_x, df_y):
    """Plots feature importance using RandomForestClassifier."""
    rfc = RandomForestClassifier()
    rfc.fit(df_x, df_y)
    importances = pd.DataFrame({'feature': df_x.columns, 'importance': rfc.feature_importances_})
    importances = importances.sort_values('importance', ascending=False).set_index('feature')
    importances.plot.bar(figsize=(12, 6), color='teal')
    plt.title("Feature Importance")
    plt.xlabel("Features")
    plt.ylabel("Importance")
    plt.show()

def plot_class_distribution(df_y):
    """Plots the distribution of classes in the dataset."""
    plt.figure(figsize=(8, 6))
    sns.countplot(x=df_y, palette="viridis")
    plt.title("Class Distribution")
    plt.xlabel("Class")
    plt.ylabel("Count")
    plt.show()

def save_to_db(df, db_name='network_intrusion.db'):
    """Saves processed dataset to SQLite database with enhanced schema."""
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Create tables with normalized schema
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS features (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                duration REAL,
                protocol_type INTEGER,
                service INTEGER,
                flag INTEGER,
                src_bytes REAL,
                dst_bytes REAL
            )
        ''')

        cursor.execute('''
            CREATE TABLE IF NOT EXISTS labels (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                feature_id INTEGER,
                class INTEGER,
                FOREIGN KEY (feature_id) REFERENCES features (id)
            )
        ''')

        # Insert data into tables
        for index, row in df.iterrows():
            cursor.execute('''
                INSERT INTO features (duration, protocol_type, service, flag, src_bytes, dst_bytes)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (row['duration'], row['protocol_type'], row['service'], row['flag'], row['src_bytes'], row['dst_bytes']))

            feature_id = cursor.lastrowid
            cursor.execute('''
                INSERT INTO labels (feature_id, class)
                VALUES (?, ?)
            ''', (feature_id, row['class']))

        conn.commit()
        print("Data saved to database successfully.")
    except Exception as e:
        print(f"Error saving data: {e}")
    finally:
        conn.close()

def main():
    """Executes the complete machine learning pipeline with enhanced database functionality."""
    train_data = load_data("/content/Train_data.csv")
    test_data = load_data("/content/Test_data.csv")

    if train_data is not None and test_data is not None:
        train_data = clean_data(train_data)
        test_data = clean_data(test_data)

        train_data = scale_features(train_data)
        test_data = scale_features(test_data)

        train_data = encode_categorical(train_data)
        test_data = encode_categorical(test_data)

        # Feature selection
        df_x = train_data.drop(['class'], axis=1)
        df_y = train_data['class']
        selected_features = feature_selection(df_x, df_y)

        # Plot class distribution
        plot_class_distribution(df_y)

        # Plot feature importance
        plot_feature_importance(df_x[selected_features], df_y)

        # Train-test split
        X_train, X_test, Y_train, Y_test = train_test_split(df_x[selected_features], df_y, train_size=0.70, random_state=42)

        # Train classifiers and display accuracy scores
        train_classifiers(X_train, Y_train, X_test, Y_test)

        # Save processed data to database
        save_to_db(train_data)

if __name__ == "__main__":
    main()